## Pandas exercise
______________________

For those of you who are interested in challenging yourselves, here's a quick exercise in manipulating data matrices into a workable format.

By default, [kallisto](https://pachterlab.github.io/kallisto/) outputs a counts matrix for each sample. This means that for each sample that you've processed you generated a separate file. What do these individual files look like? 

In [2]:
import pandas as pd

In [8]:
f = 'kallisto_counts/Control_78_Day_0_Rep3.tsv'
df = pd.read_csv(f, sep='\t') 
df.head()

,target_id,length,eff_length,est_counts,tpm
0,ENST00000456328.2,1657,1548.4200,0.078546,0.004365
1,ENST00000450305.2,632,523.4170,0.000000,0.000000
2,ENST00000488147.1,1351,1242.4200,5.916130,0.409730
3,ENST00000619216.1,68,13.5858,0.000000,0.000000
4,ENST00000469289.1,535,426.5120,0.000000,0.000000


The data in each of these files have information about expression on the **transcript level**. However we want to examine expression of **genes**, not transcripts. 

Additionally, the expression profiles of each sample are separated out by sample. We want to concatenate our matrices such that we have a **gene by sample** expression matrix. We can examine the counts matrix that was provided for us to get a rough idea of what our goal is.

In [9]:
f = 'kallisto_counts/fshd.counts.matrix.csv'
df = pd.read_csv(f, sep=',')
df.head()

,Unnamed: 0,Control_78_Day_0_Rep3,Control_78_Day_0_Rep4,Control_78_Day_3_Rep1,Control_78_Day_3_Rep2,FSHD2_19_Day_0_Rep1,FSHD2_19_Day_0_Rep2,FSHD2_19_Day_3_Rep1,FSHD2_19_Day_3_Rep2
0,ENSG00000223972.5|DDX11L1,0,2,0,0,0,0,0,0
1,ENSG00000227232.5|WASH7P,6,20,6,13,9,6,0,14
2,ENSG00000278267.1|MIR6859-1,0,0,0,0,0,0,0,0
3,ENSG00000243485.5|MIR1302-2HG,0,0,0,0,0,0,0,0
4,ENSG00000284332.1|MIR1302-2,0,0,0,0,0,0,0,0


Finally I will provide one more resource. We need a way to map transcript IDs to gene IDs. This file is provided at `/pub/erebboah/cosmos/FSHD_bulkRNA/ref/hg38.gene_transcripts.tab`. This is what the file looks like. Here, 'tid' stands for transcript ID, 'gname' stands for gene name, and 'gid' stands for gene ID.

In [11]:
f = 'ref/hg38.gene_transcripts.tab'
df = pd.read_csv(f, sep='\t', header=None, 
                 usecols=[0,1,2], 
                 names=['gid', 'gname', 'tid'])
df.head()

,gid,gname,tid
0,ENSG00000223972.5,DDX11L1,ENST00000456328.2
1,ENSG00000223972.5,DDX11L1,ENST00000450305.2
2,ENSG00000227232.5,WASH7P,ENST00000488147.1
3,ENSG00000278267.1,MIR6859-1,ENST00000619216.1
4,ENSG00000243485.5,MIR1302-2HG,ENST00000469289.1


A few closing notes:
* use the `est_counts` column from the kallisto output files 
* you can either use gene ID or gene name (or both!) as your gene identifier, gene IDs are typically more reliable though because there are some sneaky duplicate gene IDs for different genes 💀

Best of luck!

## Solution 1
------------------------------------------

In [32]:
import glob
import os
import time

In [36]:
# time it just for fun
start = time.time()

# get a list of the filenames we'll concatenate
files = glob.glob('kallisto_counts/*tsv')

# read in the gene ID to transcript ID reference
ref = 'ref/hg38.gene_transcripts.tab'
ref_df = pd.read_csv(ref, sep='\t', header=None, 
                 usecols=[0,1,2], 
                 names=['gid', 'gname', 'tid'])

# loop through each file
for i, f in enumerate(files):
    curr_df = pd.read_csv(f, sep='\t')
    
    # rename columns; specifically the counts column
    # to be the name of the sample 
    sample = os.path.basename(os.path.splitext(f)[0])
    curr_df.rename({'target_id':'tid', 'est_counts': sample}, axis=1, inplace=True)
    
    # merge with gene information
    curr_df = curr_df.merge(ref_df, how='left', on='tid')
    
    # limit to just the columns we want
    curr_df = curr_df[['gid', 'gname', 'tid', sample]]
    
    # if this is the first file we're opening, just make a new variable 
    if i == 0:
        df = curr_df
    # otherwise merge with the preexisting df
    else:
        df = df.merge(curr_df, on=['gid', 'gname', 'tid'])

# then groupby gene ID and gene name and sum
df.drop('tid', axis=1, inplace=True)
df = df.groupby(['gid', 'gname']).sum().reset_index()

# finally save to a new file
f = 'full_fshd_counts.tsv'
df.to_csv(f, sep='\t', index=False)

print("--- %s seconds ---" % (time.time() - start))

--- 24.774352073669434 seconds ---


In [37]:
df.head()

,gid,gname,FSHD2_19_Day_0_Rep1,FSHD2_19_Day_3_Rep2,Control_78_Day_3_Rep2,Control_78_Day_3_Rep1,FSHD2_19_Day_0_Rep2,FSHD2_19_Day_3_Rep1,Control_78_Day_0_Rep3,Control_78_Day_0_Rep4
0,ENSG00000000003.14,TSPAN6,539.913650,603.312470,666.193870,462.04010,906.87130,525.63242,903.993050,901.777200
1,ENSG00000000005.5,TNMD,0.000000,0.000000,1.000000,0.00000,0.00000,0.00000,0.000000,0.000000
2,ENSG00000000419.12,DPM1,436.999800,562.000500,816.000400,556.00006,530.99990,486.00022,783.999600,719.999800
3,ENSG00000000457.13,SCYL3,71.419568,100.746300,145.600540,76.83710,88.90650,68.07656,129.608058,128.507010
4,ENSG00000000460.16,C1orf112,181.646110,46.838622,103.611302,90.33070,206.65181,47.01866,226.485600,270.624511
